## Table of Contents

- [Y2 Model Based Definition](#Y2-Model-Based-Definition)
  - [2.1 Introduction](#2.1-Introduction)
  - [2.3 User Stories:](#2.3-User-Stories%3A)
- [🏠 Home](../../welcomePage.ipynb)

# Y2 Model Based Definition
## 2.1 Introduction
Model-based definition (MBD) is a process that uses 3D models within CAD software to define product assemblies and individual components comprehensively. The model is annotated with non-geometric information, such as dimensions, materials, finishes, tolerances, and symbols, that are important for manufacturing and inspection. The model becomes the source of authority for all engineering activities and is used in downstream processes like manufacturing, inspection, and assembly. MBD can offer several benefits, including: (a) Reduced engineering documentation time: Compared to drawing-centric workflows, MBD can reduce the time spent on engineering documentation, (b) More efficient drafting process: When designers designate critical dimensions, weld callouts, and notes in the model before passing it on to drafters, there's less back-and-forth between the two, allowing drafters to have all the information they need as soon as it's ready, (c) Improved communication and collaboration: MBD can improve communication and collaboration among stakeholders, and (d) Cost savings: efficiencies gained through MBD lead to substantial cost savings.

<center>
<img src="../../img/05.jpg" alt="Alt text" width="500"></center>

### 2.2 Concepts:
- **Conceptual Model:** A conceptual model is an abstract representation of a system or process, designed to help understand, predict, or communicate its essential features. It often involves diagrams, flowcharts, or other visual tools to outline the relationships and interactions within the system, without diving into detailed specifics.

- **Fully Defined Model:** A fully defined model includes all the specific details necessary for implementation or analysis. In engineering and design, this means all dimensions, materials, tolerances, and other parameters are specified, allowing for accurate manufacturing, simulation, or evaluation.

- **CAD/Scanning Tools:** CAD (Computer-Aided Design) tools are software applications used to create, modify, analyze, or optimize a design. These tools enable the creation of detailed 2D or 3D models of physical objects, which can be used for engineering, manufacturing, and simulation purposes. 
    
    Scanning tools, often used in conjunction with CAD, capture the physical dimensions and features of real-world objects to create precise digital representations.

    <img src="../../img/17.jpg" alt="Alt text" width="300">

- **CAM Tools:** CAM (Computer-Aided Manufacturing) tools use CAD data to control and automate manufacturing processes. These tools generate the instructions (such as G-code) that guide machinery in fabricating parts, ensuring precision and efficiency in production. CAM tools integrate design and manufacturing processes, facilitating the transition from digital models to physical products.

    <img src="../../img/18.jpg" alt="Alt text" width="300">
    
<center>
<img src="../../img/Model Based Definition.svg" alt="Alt text" width="500"></center>

## 2.3 User Stories:
#### The "M" Block


<center> <img src="../../img/Animation2.gif" alt="Alt text" width="60%"> </center>

Standard CAD formats can store data such as dimentions, tolerance, material, and manufacturing method. Projects using a model based definition need to store more information than what standard CAD formats allow. This 'M' block model was designed for a model based definition. It contains the standard CAD information such as dimentions, tolerance, material, and manufacturing method; but it also contains information about the manufacturing process such as the maximum recored deviation or final dimentions. It also contains sensor information from the manufacturing process, such as motor current feedback, and 3D scan data. It can also store data for non engineering needs such as final cost, packaging requirements, and final customer.

To achieve this we created our own file format (.mbe) that stores all this data. The benefit of this format is you can load a specific part of the file when needed. If the engineers need to view the 3d scan, they open the .mbe file and the program will only load the 3D scan it wont load the other information. If sales people need to view the final cost they can select the .mbe file and their program will only load the infromation necessary for sales. The .mbe file is the authoritative source of the product defiition.

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
from PIL import Image
from fpdf import FPDF
from base64 import b64encode
import os
import io
import json  # PyMuPDF
# import myFunctions as myFn


# Create functions to be used later
def setYpos(currentSection, lastYpos, currentYpos, is_LR):
    lastYpos[is_LR] = currentYpos
    return lastYpos

def printPositionDebug(pos_debug, lastYpos, currentYpos, curretSection, sectionName, is_LR):
    LR_def = ["Left", "Right"]

    if pos_debug == 1:
        print("")
        print(f"🏃 Position Debug -- Section: {sectionName}")
        print(f"Current Section: {curretSection}")
        print(f"Current Side: {LR_def[is_LR]}")
        print(f"End Y Position: {currentYpos}")
        print(f"Last Y Position\n   Left: {lastYpos[0]}  |  Right: {lastYpos[1]}")
        print("")

def list_mbd_subfolders(parent_folder):
    """
    Lists all first-layer subfolders in the specified directory that have '.mbd' in their names.

    Args:
        parent_folder (str): Path to the parent folder.

    Returns:
        list: A list of subfolder paths with '.mbd' in their names.
    """
    if not os.path.isdir(parent_folder):
        print(f"The provided path '{parent_folder}' is not a valid directory.")
        return []

    # Get all first-layer subdirectories
    subfolders = [
        # os.path.join(parent_folder, subfolder)
        subfolder
        for subfolder in os.listdir(parent_folder)
        if os.path.isdir(os.path.join(parent_folder, subfolder)) and '.mbd' in subfolder
    ]

    return subfolders

# Set File Locations
MBD_Location = "../../example_resources/MBD_Files"
mbdFileList = list_mbd_subfolders(MBD_Location)

# Create a text box
text_box = widgets.Textarea(
    value='',
    placeholder='Type notes...',
    description='Notes:',
    disabled=False
)

# Variable to store the resulting text
resulting_text = ''

def update_text(change):
    global resulting_text
    resulting_text = change['new']
    # print(f"Updated Text: {resulting_text}")

# Attach the update function to the text box
text_box.observe(update_text, names='value')

# Create a dropdown menu
dropdown = widgets.Dropdown(
    options=mbdFileList,
    value=mbdFileList[0],
    description='MBD File:',
    disabled=False
)

# Variable to store the selected dropdown value
selected_option = mbdFileList[0]
def update_selection(change):
    global selected_option    
    selected_option = change['new']
    # print(f"Option selected: {selected_option}")

# Attach the update function to the dropdown
dropdown.observe(update_selection, names='value')

# Display the widget
display(dropdown, text_box)

Dropdown(description='MBD File:', options=('iceCleat.mbd', 'mBlock.mbd'), value='iceCleat.mbd')

Textarea(value='', description='Notes:', placeholder='Type notes...')

In [6]:
# A4 Dimentions
w = 210
h = 297

notes = resulting_text
outputFile = selected_option.replace(".mbd", ".pdf")

# partFile = "mBlock.mbd"
# partFile = "iceCleat.mbd"

partFile = MBD_Location + "/" + selected_option

# Import MBD Data
with open(f"{partFile}/info.json", "r") as file:
    partData = json.load(file)

# Setup Document Vairables
docFont = "dejavu-sans"
verticalSectionSpacing = 10
cellWidth = (w-20)/2-10
cellHeight = 8
smallSpacerHeight = 3

xPos = (10, (w-20)/2+20)
lastYpos = (0,0)

currentSection = 0

try:
    partName = partData["general_properties"]["name"]
    partID = partData["general_properties"]["part_id"]
except:
    print(f"🛑 Part Name or Part ID Missing 🛑")

header_start_page = 1
footer_start_page = 0

# Debug Enables
border_enable = False
pos_debug = False
column_choice_debug = False

output_real_file = False

# Setup FPDF Objects
class FPDF(FPDF):
    
    # Header
    def header(self):
        pageNum = self.page_no()
        if pageNum > header_start_page:
            self.set_font(docFont, "", 10)
            self.cell(0,5,f"{partID} [{partName}]", border=border_enable, align="R")
            self.image("example_resources/Images/UM-Dearborn_Logo-horizontal-RGB.png", h=5, x=10)
            self.set_xy(10,30)
            
    # Footer
    def footer(self):
        pageNum = self.page_no()
        footer_height = 5
        footer_font_size = 10

        if pageNum > footer_start_page:
            self.set_font(docFont, "", footer_font_size)
            self.set_y(-10-footer_height)
            self.cell((w-20)/3, footer_height,f"Page: {str(pageNum-footer_start_page)}", border=border_enable)
            self.cell((w-20)/3, footer_height,f"{partID} [{partName}]", border=border_enable, align="C")
            self.cell((w-20)/3, footer_height,"Data Sheet", border=border_enable, align="R")
    
    # Section Title
    def section_title(self, text, spacer_below_height):
        self.set_fill_color(0, 39, 76)
        self.set_font(docFont, "", 14)
        global is_LR

        # Determine if new cell should be left or right
        if lastYpos[0] == lastYpos[1]:
            self.set_xy(xPos[currentSection % 2], lastYpos[currentSection % 2])
            is_LR = currentSection % 2

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print(f'💈 Column Choice Debug -- Section: {text}')
                print(f'Columns are Identical in Length Chose: {currentSection % 2} out of {lastYpos}')
                print("")
        else:
            shortestColumn = lastYpos.index(min(lastYpos))
            self.set_xy(xPos[shortestColumn], lastYpos[shortestColumn])
            is_LR = shortestColumn

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print(f'💈 Column Choice Debug -- Section: {text}')
                print(f'Column {shortestColumn} is the shortest at {lastYpos[shortestColumn]} out of {lastYpos}')
                print("")
        
        # Set Font Settings for Section Title
        self.set_text_color(255, 202, 5)
        self.cell(cellWidth, cellHeight, "", border=border_enable, new_x="LEFT", new_y="NEXT")
        self.cell(cellWidth, cellHeight, text, fill=True, align="C", border=border_enable, new_x="LEFT", new_y="NEXT")
        
        # Spacer below section title
        self.cell(cellWidth, spacer_below_height, "", border=border_enable, new_x="LEFT", new_y="NEXT")
        
        # Reset Font for normal text
        self.set_text_color(0, 0, 0)
        self.set_font(docFont, "", 12)

    # Empty Section Title (For images and other non text based sections)
    def no_section_title(self):
        global is_LR
        self.set_fill_color(0, 39, 76)
        self.set_text_color(255, 202, 5)
        
        # Determine if new cell should be left or right
        if lastYpos[0] == lastYpos[1]:
            self.set_xy(xPos[currentSection % 2], lastYpos[currentSection % 2])
            is_LR = currentSection % 2

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print("💈 Column Choice Debug -- Titleless Section")
                print(f'Columns are Identical in Length Chose: {currentSection % 2} out of {lastYpos}')
                print("")
        else:
            shortestColumn = lastYpos.index(min(lastYpos))
            self.set_xy(xPos[shortestColumn], lastYpos[shortestColumn])
            is_LR = shortestColumn

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print("💈 Column Choice Debug -- Titleless Section")
                print(f'Column {shortestColumn} is the shortest at {lastYpos[shortestColumn]} out of {lastYpos}')
                print("")
            
        self.cell(cellWidth, cellHeight, "", border=border_enable, new_x="LEFT", new_y="NEXT") # Blank Cell

# Enable FPDF
pdf = FPDF("P", "mm", format=(w,h))

# Setup Fonts and Fallback Fonts
pdf.add_font(fname="../../example_resources/Fonts/TwitterColorEmoji-SVGinOT.ttf")
pdf.add_font(family="dejavu-sans", style="", fname="../../example_resources/Fonts/DejaVuSans.ttf")
pdf.add_font(family="dejavu-sans", style="b", fname="../../example_resources/Fonts/DejaVuSans-Bold.ttf")
pdf.set_fallback_fonts(["TwitterColorEmoji-SVGinOT", "dejavu-sans"])

pdf.set_font(docFont, "", 16)

## Page 1
pdf.add_page()

## Title
title_string = f"{partID} [{partName}]"
title_font_size = 25
pdf.set_text_color(0, 39, 76)
pdf.set_font(docFont, "b", title_font_size)

# Check to see how long title is and change font size accordingly
if pdf.get_string_width(title_string) <= 140:
    pdf.set_x(10)
elif pdf.get_string_width(title_string) >= (w-40):
    pdf.set_x(30)
    title_length = pdf.get_string_width(title_string)
    # Find new font size to make sure it will fit in title block
    while title_length >= (w-40):
        title_font_size += -1
        pdf.set_font(docFont, "b", title_font_size)
        title_length = pdf.get_string_width(title_string)
        if title_font_size <= 12.5:
            break
else:
    pdf.set_x(30)
    
pdf.multi_cell(0, 25/2, title_string, border=border_enable, align="C", max_line_height=title_font_size/2)

## Subtitle
pdf.set_font(docFont, "", 18)
pdf.set_y(21)
pdf.cell(0,9,"Data Sheet", border=border_enable, align="C", new_x="LEFT", new_y="NEXT")
# UMDearborn Logo
image = "../../example_resources/Images/UM-Dearborn_Logo-RGB.png"
pdf.image(image, h=20 , x=10, y=10)

lastYpos = [pdf.get_y(),pdf.get_y()]

pdf.set_text_color(0, 0, 0)
pdf.set_x(10)

## Sedctions

# Applications Secion
try:
    partApplications = partData["general_properties"]["applications"]
except:
    print("⚠️ general_properties: appliacations not found, skipping ⚠️")
else:
    pdf.section_title("Applications", smallSpacerHeight)

    text = ""

    for i in partApplications:
        if i == partApplications[0]:
            text = text + "  -  " + i
        else:
            text = text + "\n" + "  -  " + i  
        
    pdf.multi_cell(cellWidth, 6, text, border=border_enable, new_x="LEFT", new_y="NEXT")
    
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Applications", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Image
try:
    partImage = partData["general_properties"]["image"]
except:
    print("⚠️ general_properties: image not found, skipping ⚠️")
else:
    pdf.no_section_title()

    pdf.image(f"{partFile}/{partImage}", w=(w-20)/2-30, x = xPos[currentSection % 2]+10)

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Image", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Description
try:
    partDescription = partData["general_properties"]["description"]
except:
    print("⚠️ gneral_properties: description not found, skipping ⚠️")
else:
    pdf.section_title("Description", smallSpacerHeight)
    
    pdf.multi_cell(cellWidth, 5, partDescription, align="J", border=border_enable)

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Description", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Physical Properties
try:
    partPhysicalProperties = partData["physical_properties"]
except:
    print("⚠️ physical_properties not found, skipping ⚠️")
else:
    pdf.section_title("Physical Properties", smallSpacerHeight)

    # Material
    try:
        partMaterial = partPhysicalProperties["material"]
    except:
        print("⚠️ Part Material Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Material = {partMaterial}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Mass
    try:
        partMass = partPhysicalProperties["mass"]
        partMassUnit = partPhysicalProperties["mass_unit"]
    except:
        print("⚠️ Part Mass or Mass Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Mass = {partMass} [{partMassUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Volume
    try:
        partVolume = partPhysicalProperties["volume"]
        partVolumeUnit = partPhysicalProperties["volume_unit"]
    except:
        print("⚠️ Part Volume or Volume Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Volume = {partVolume} [{partVolumeUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Hardness
    try:
        partHardness = partPhysicalProperties["hardness"]
        partHardnessUnit = partPhysicalProperties["hardness_unit"]
    except:
        print("⚠️ Part Hardness or Hardness Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Hardness = {partHardness} [{partHardnessUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")
    
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Physical Properties", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1


# Manufacturing Properties
try:
    partManufacturingProperties = partData["manufacturing_properties"]
except:
    print("⚠️ manufacturing_properties not found, skipping ⚠️")
else:
    pdf.section_title("Maufacturing Properties", smallSpacerHeight)

    # Method
    try:
        manufacturingMethod = partManufacturingProperties["method"]
    except:
        print("⚠️ Manufacturing Method Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Process = {manufacturingMethod}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Waste Percent
    try:
        manufacturingWaste = partManufacturingProperties["waste_percent"]
    except:
        print("⚠️ Manufacturing Waste Percent Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Waste = {manufacturingWaste}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Scrap Rate
    try:
        manufacturingScrap = partManufacturingProperties["scrap_rate"]
    except:
        print("⚠️ Manufacturing Scrap Rate Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Scrap = {manufacturingScrap}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Hardness
    try:
        manufacturingCarbonFootprint = partManufacturingProperties["carbon_footprint"]
        manufacturingCarbonFootprintUnit = partManufacturingProperties["carbon_footprint_unit"]
    except:
        print("⚠️ Carborn Footprint or Carbon Footprint Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Carbon Footprint = {manufacturingCarbonFootprint} [{manufacturingCarbonFootprintUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")
   
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Maunfacturing Properties", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1


# Notes
try:
    notes = notes
except:
    print("😡 You messed with the code 😡\n😡 The notes are missing 😡")
else:
    pdf.section_title("Notes", smallSpacerHeight)
    pdf.multi_cell(cellWidth, 5, notes, border=border_enable, align="L")

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Notes", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# # Add Second Page
# pdf.add_page()

# Output PDF file and write file to variable for display
pdf_bytes = pdf.output()
print(f"✅ PDF Generation Successful")

if output_real_file == True:
    pdf.output(outputFile)
    print(f"✅ Output Location: './{outputFile}'")

# New PDF Display
DisplayWidth, DisplayHeight = 800, 750
base64_pdf = b64encode(pdf_bytes).decode("utf-8")
display(HTML(f'<embed height="{DisplayHeight}" src="data:application/pdf;base64,{base64_pdf}" type="application/pdf" width="{DisplayWidth}"/>'))

# # Old PDF Display
# pdf_path = outputFile  # Replace with your PDF file path
# viewer = myFn.PDFViewer(pdf_path)
# viewer.show()

✅ PDF Generation Successful


### <center>[◀︎ Module 1](Module1.ipynb)     [🏠 Home](../../welcomePage.ipynb)     [Module 3 ▶︎](Module3.ipynb)</center>